# Hackaton project

Идея проекта - оповещать персонал мед клиники о местонахождении маломобильного человека. 

Датасет взят из открытых источников, с сайта https://universe.roboflow.com/

Модель распознавания - YOLO 8 версия

Для оповещения сделан бот в телеграме, изначально он закомментирован (последняя строка), раскомментируйте его, потом запускайте код.

## Грузим необходимые библиотеки и зависимости

In [1]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [2]:
from ultralytics import YOLO
import os
from PIL import Image

### Загружаем модель

In [3]:
model = YOLO('yolov8m.pt')

In [6]:
model.compile()

### Обучаем модель

Сделано 10 эпох и оптимизатор - стохастический градиентный спуск (SGD)

In [4]:
model.train(data = 'datasets/data.yaml', epochs = 10, optimizer = 'SGD')

Ultralytics 8.3.51  Python-3.12.4 torch-2.5.1+cpu CPU (AMD Ryzen 5 5600H with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=datasets/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_

train: Scanning C:\Users\kat_y\Documents\Python projects (clouds edition)\MIPT\hackaton\datasets\train\labels.cache... 
val: Scanning C:\Users\kat_y\Documents\Python projects (clouds edition)\MIPT\hackaton\datasets\valid\labels.cache... 76


Plotting labels to runs\detect\train5\labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train5
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/18 [00:02<?, ?it/s]


KeyboardInterrupt: 

## Тестируем модель на новых данных

Грузим новую модель, лучшую из всех, которые были получены на предыдущем этапе

In [32]:
new_model = YOLO('../runs/detect/train42/weights/best.pt')

Грузим картинки из папки, делаем предсказание.

In [76]:
# Укажите путь к папке с картинками
image_folder = 'datasets/test/images'

# Получаем список файлов в папке
image_files = os.listdir(image_folder)

results = []

# Итерируемся по файлам и загружаем картинки
for image_file in image_files:
    # Формируем полный путь к файлу
    image_path = os.path.join(image_folder, image_file)
    # Загружаем картинку
    image = Image.open(image_path)
    # Теперь вы можете работать с загруженной картинкой
    print(f'Загружена картинка: {image_file}')
    # Делаем предсказание для каждой картинки
    new_model.predict(image)
    print('\n')    
    # Здесь можно добавить проверку на качество предсказания или ещё что-то, но пока ее нет
    results.append(image_path)
    # Закрываем картинку
    image.close()

Загружена картинка: 001675b1bb8d54ed_jpg.rf.c51966d1a9f74710b62979c0262c0135.jpg

0: 640x640 1 Wheelchair, 281.1ms
Speed: 5.1ms preprocess, 281.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


Загружена картинка: 001675b1bb8d54ed_jpg.rf.f0e904cbc862a21776b9c501fa20a49b.jpg

0: 640x640 (no detections), 270.3ms
Speed: 0.0ms preprocess, 270.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


Загружена картинка: 008b6a82d34b59c5-Copy_jpg.rf.94c77ba9c6b915e69ec4705d8639acd9.jpg

0: 640x640 3 Wheelchairs, 276.3ms
Speed: 1.9ms preprocess, 276.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


Загружена картинка: 023dac604c016ac3-Copy_jpg.rf.3f61dd7b150550c8462a3caf2d82837a.jpg

0: 640x640 1 Wheelchair, 283.1ms
Speed: 2.2ms preprocess, 283.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


Загружена картинка: 024cd3c930845976-Copy_jpg.rf.259c8254f15868a8f2f2af1659a0d49b.jpg

0: 640x640 (no detections), 272.7ms
Spe

In [88]:
#print(results) # это проверка для самой себя

## Импортируем нужные библиотеки и зависимости для бота

In [143]:
pip install pytelegrambotapi

Note: you may need to restart the kernel to use updated packages.


In [158]:
pip install python-telegram-bot

Note: you may need to restart the kernel to use updated packages.


In [89]:
import telebot
from telebot import types

## Собственно инициализация бота, токен записан в отдельном файле token.txt

In [83]:
# reading the token
with open('token.txt', 'r') as file:
    token = file.read().strip()

# our bot init
bot = telebot.TeleBot(token)

Очень простой вариант ответа на сообщения пользователя, чтобы было. 

Пишем отправку сообщений, картинка и текст - обнаружен маломобильный пользователь. В будущем здесь добавить бы в сообщение номер камеры, на которой был замечен человек, или место, где человек находится. Ну и адекватно сделать оповещение, пока что оно костыльно сделано

In [84]:
@bot.message_handler(content_types=['text'])
def get_text_messages(message):
    chat_id = message.chat.id
    if message.text in ["ок", "Ок", "ОК", "оК"]:
        bot.send_message(chat_id, "Отлично, ответ получен.")
    elif message.text in ["привет", "ПРивет", "Привет", "ПРИВЕТ"]:
        bot.send_message(chat_id, "Привет! Я бот, оповещающий о наличии маломобильных людей на территории больницы.")
    elif message.text == "есть новости":
        bot.send_photo(chat_id, photo=open(results[3], 'rb'), caption='Обнаружен маломобильный пользователь')
    elif message.text == "/help":
        bot.send_message(chat_id, "Напиши привет")
    elif message.text == "/start":
        bot.send_message(chat_id, "Привет, новый пользователь! Я бот, оповещающий о наличии маломобильных людей на территории больницы.")
    else:
        bot.send_message(chat_id, "Я тебя не понимаю. Напиши /help.")

## Следующая ячейка запустит бота @low_mob_alert_bot в телеграмме

Бот не остановится, пока пользователь специально не прервет работу ядра.
Пожалуйста, осторожнее с этим куском кода. 
Раскомментируйте его перед запуском

In [86]:
#bot.polling(non_stop=True)